In [4]:
import os
os.chdir('/Users/alexascunceparis/Desktop/BSC/immuno_project/TCRranker')

In [1]:
#Import libraries
import os
import pandas as pd
import numpy as np
from Bio import PDB


residue_mapping = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
    'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
    'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
    'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
}

def extract_contacts(pdb_files, chain_dict, distance=5):
    """
    Extract contacting atoms between specific chains based on a distance threshold.
    
    Args:
        pdb_files (list of str): List of file paths to PDB files.
        chain_dict (dict): Dictionary containing chains for each PDB ID.
        distance (float): Distance threshold for considering contacts.
    
    Returns:
        pd.DataFrame: DataFrame containing contacts with columns ['pdb_id', 'chain_from', 'chain_to', 
                                                                 'residue_from', 'residue_to', 
                                                                 'resid_from', 'resid_to', 
                                                                 'atom_from', 'atom_to', 'dist'].
    """
    contacts = []
    
    for pdb_file in pdb_files:
        pdb_id = os.path.basename(pdb_file).split('.')[0]
        print(f"Extracting contacts from {pdb_id}")
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_id, pdb_file)
        model = structure[0]

        # Retrieve chains of interest from chain_dict
        chains = chain_dict.get(pdb_id)
        if not chains:
            continue
        
        # Define pairs of chains to extract contacts
        chain_pairs = [
            (chains['tcra_chain'], chains['mhc_chain']),
            (chains['tcrb_chain'], chains['mhc_chain']),
            (chains['tcra_chain'], chains['peptide_chain']),
            (chains['tcrb_chain'], chains['peptide_chain'])
        ]
        
        for chain_from_id, chain_to_id in chain_pairs:
            if chain_from_id and chain_to_id:  # Ensure both chains are defined
                try:
                    chain_from = model[chain_from_id]
                    chain_to = model[chain_to_id]
                except KeyError:
                    print(f"Chain not found in {pdb_id}: {chain_from_id} or {chain_to_id}")
                    continue
                
                residues_from = list(chain_from.get_residues())
                residues_to = list(chain_to.get_residues())
                
                for residue_from in residues_from:
                    for residue_to in residues_to:
                        atoms_from = list(residue_from.get_atoms())
                        atoms_to = list(residue_to.get_atoms())
                        
                        for atom_from in atoms_from:
                            for atom_to in atoms_to:
                                dist = np.linalg.norm(atom_from.coord - atom_to.coord)
                                if dist <= distance:  # Threshold for contact
                                    res_from_single = residue_mapping.get(residue_from.get_resname(), 'X')  # 'X' for unknown residues
                                    res_to_single = residue_mapping.get(residue_to.get_resname(), 'X')  # 'X' for unknown residues
                                    
                                    # Original contact
                                    contacts.append([
                                        pdb_id, 
                                        chain_from.get_id(), 
                                        chain_to.get_id(), 
                                        res_from_single, 
                                        res_to_single, 
                                        residue_from.get_id()[1], 
                                        residue_to.get_id()[1], 
                                        atom_from.get_id(), 
                                        atom_to.get_id(), 
                                        dist
                                    ])
    
    return pd.DataFrame(contacts, columns=['pdb_id', 'chain_from', 'chain_to', 'residue_from', 'residue_to', 'resid_from', 'resid_to', 'atom_from', 'atom_to', 'dist'])

In [11]:
# Directorio que contiene los archivos PDB
pdb_dir = './pdb_nr/'  # Cambié la ruta a ./pdb_nr/ como indicaste
pdb_files = [os.path.join(pdb_dir, f) for f in os.listdir(pdb_dir) if f.endswith('.pdb')]

chain_dict = {}
general_df = pd.read_csv('./structures_annotation/general.txt', sep='\t')

# Crear un diccionario de cadenas basado en la información del archivo general
for pdb_id, group in general_df.groupby('pdb.id'):
    chains = {
        'tcra_chain': None,
        'tcrb_chain': None,
        'peptide_chain': None,
        'mhc_chain': None
    }
    
    for _, row in group.iterrows():
        if row['chain.component'] == 'TCR' and row['chain.type'] == 'TRA':
            chains['tcra_chain'] = row['chain.id']
        elif row['chain.component'] == 'TCR' and row['chain.type'] == 'TRB':
            chains['tcrb_chain'] = row['chain.id']
        elif row['chain.component'] == 'PEPTIDE':
            chains['peptide_chain'] = row['chain.id']
        elif row['chain.component'] == 'MHC' and row['chain.supertype'] == 'MHCI' and row['chain.type'] == 'MHCa':
            chains['mhc_chain'] = row['chain.id']
        
    chain_dict[pdb_id] = chains 

# Extraer contactos
for pdb_file in pdb_files:
    pdb_id = os.path.basename(pdb_file).split('.')[0]
    contacts_df = extract_contacts([pdb_file], chain_dict)

    # Guardar los resultados en un archivo CSV
    output_file = f'./contact_maps/{pdb_id}_contacts.csv'
    if os.path.exists(output_file):
        print(f"El archivo {output_file} ya existe, omitiendo...")
        continue
    contacts_df.to_csv(output_file, index=False)
    print(f"Contactos extraídos y guardados en {output_file}.")

Extracting contacts from 5m02
Contactos extraídos y guardados en ./contact_maps/5m02_contacts.csv.
Extracting contacts from 8cx4
Contactos extraídos y guardados en ./contact_maps/8cx4_contacts.csv.
Extracting contacts from 5tez
